In [1]:
import time
#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split
from math import *
from sklearn import metrics
#
pd.set_option('display.max_columns', 500)  # force pandas to display all columns for better visual inspection
# plot plots inline
%matplotlib inline 

In [2]:
trn_all = pd.read_csv('../data/train.csv')  # column #0 in our file is index
# trn_1 = pd.read_csv('../data/train_1.csv', index_col=0)

In [3]:
#selected features and Cut off outliers of Expected >= 69
index=list(trn_all)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18, 23]
trn_new = trn_all[[index[i] for i in my_indices]]

trn_new = trn_new[trn_new['Expected']<69]

In [10]:
trn_new

,Id,minutes_past,radardist_km,Ref,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_50th,RefComposite_5x5_90th,Zdr,Zdr_5x5_50th,Zdr_5x5_90th,Expected
0,1,3,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
1,1,16,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
2,1,25,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
3,1,35,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
4,1,45,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
5,1,55,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.254000
6,2,1,2,9.0,7.5,10.5,15.0,16.5,23.5,0.3750,0.3125,0.8750,1.016000
7,2,6,2,26.5,25.5,31.5,26.5,28.5,32.0,0.0625,0.2500,0.6875,1.016000
8,2,11,2,21.5,20.5,25.0,26.5,25.0,27.0,0.3125,0.3125,0.6250,1.016000
9,2,16,2,18.0,17.5,21.0,20.5,20.5,23.0,0.2500,0.3750,0.6875,1.016000


In [4]:
#Cut off Ref values < 0
#trn_new[trn_new['Ref_5x5_50th']<0] = np.nan
#trn_new[trn_new['Ref_5x5_90th']<0] = np.nan
#trn_new[trn_new['RefComposite']<0] = np.nan
#trn_new[trn_new['RefComposite_5x5_50th']<0] = np.nan
#trn_new[trn_new['RefComposite_5x5_90th']<0] = np.nan
#trn_new[trn_new['Ref']<0] = np.nan

In [5]:
#combine observations with same ID by using mean
#replace Nan by overall mean
trn_mean = trn_new.groupby(trn_new.Id).agg(['mean', 'median', 'std', 'count'])
trn_mean.columns = ['_'.join(col).strip() for col in trn_mean.columns.values]

In [6]:
index2 = list(trn_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)



trn_mean= pd.DataFrame(imp.fit_transform(trn_mean),index = trn_mean.index, columns=index2)

In [8]:
trn_mean

,minutes_past_mean,minutes_past_median,minutes_past_std,minutes_past_count,radardist_km_mean,radardist_km_median,radardist_km_std,radardist_km_count,Ref_mean,Ref_median,Ref_std,Ref_count,Ref_5x5_50th_mean,Ref_5x5_50th_median,Ref_5x5_50th_std,Ref_5x5_50th_count,Ref_5x5_90th_mean,Ref_5x5_90th_median,Ref_5x5_90th_std,Ref_5x5_90th_count,RefComposite_mean,RefComposite_median,RefComposite_std,RefComposite_count,RefComposite_5x5_50th_mean,RefComposite_5x5_50th_median,RefComposite_5x5_50th_std,RefComposite_5x5_50th_count,RefComposite_5x5_90th_mean,RefComposite_5x5_90th_median,RefComposite_5x5_90th_std,RefComposite_5x5_90th_count,Zdr_mean,Zdr_median,Zdr_std,Zdr_count,Zdr_5x5_50th_mean,Zdr_5x5_50th_median,Zdr_5x5_50th_std,Zdr_5x5_50th_count,Zdr_5x5_90th_mean,Zdr_5x5_90th_median,Zdr_5x5_90th_std,Zdr_5x5_90th_count,Expected_mean,Expected_median,Expected_std,Expected_count
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,29.833333,30.0,19.104101,6,10,10,0,6,21.405729,21.380878,6.204164,0,21.189465,21.298599,5.525985,0,24.228192,24.35923,6.132682,0,22.965544,22.941273,6.224182,0,22.807069,22.902906,5.639536,0,25.519516,25.609721,6.232449,0,0.590932,0.546352,1.166220,0,0.225834,0.254181,0.527364,0,1.983693,1.909905,1.047196,0,0.254000,0.254000,0,6
2,29.083333,28.5,18.725084,12,2,2,0,12,16.625000,17.000000,7.296092,12,17.375000,18.250000,5.792334,12,21.333333,22.25000,6.231640,12,22.666667,22.500000,5.166911,12,22.916667,22.500000,4.072264,12,25.916667,25.250000,3.930263,12,0.380208,0.375000,0.240528,12,0.416667,0.375000,0.203543,12,0.781250,0.687500,0.215685,12,1.016001,1.016000,0,12
3,30.750000,30.5,17.689879,12,10,10,0,12,21.405729,21.380878,6.204164,0,21.189465,21.298599,5.525985,0,8.500000,8.50000,6.132682,1,22.965544,22.941273,6.224182,0,22.807069,22.902906,5.639536,0,8.500000,8.500000,6.232449,1,0.590932,0.546352,1.166220,0,0.225834,0.254181,0.527364,0,2.062500,2.062500,1.047196,1,26.162014,26.162014,0,12
4,28.153846,28.0,18.297751,13,9,9,0,13,26.600000,25.750000,9.091510,10,25.800000,25.250000,9.758757,10,30.269231,30.00000,11.450283,13,26.666667,25.000000,11.368484,12,25.115385,23.500000,10.749329,13,32.230769,33.500000,10.599952,13,-1.125000,-1.125000,1.166220,1,0.500000,0.500000,0.527364,1,1.515625,1.000000,1.385730,4,4.064002,4.064002,0,13
6,29.833333,30.0,19.104101,6,13,13,0,6,21.405729,21.380878,6.204164,0,21.189465,21.298599,5.525985,0,24.228192,24.35923,6.132682,0,22.965544,22.941273,6.224182,0,22.807069,22.902906,5.639536,0,25.519516,25.609721,6.232449,0,0.590932,0.546352,1.166220,0,0.225834,0.254181,0.527364,0,1.983693,1.909905,1.047196,0,0.254000,0.254000,0,6
7,30.933333,31.0,17.778264,15,13,13,0,15,14.750000,14.750000,0.353553,2,12.500000,12.500000,1.683251,4,14.428571,14.00000,3.610501,7,14.750000,14.500000,1.707825,4,14.875000,14.500000,2.056494,4,15.714286,16.000000,3.828154,7,0.590932,0.546352,1.166220,0,0.225834,0.254181,0.527364,0,1.983693,1.909905,1.047196,0,0.508000,0.508000,0,15
8,31.083333,31.5,18.470903,12,8,8,0,12,19.600000,20.500000,6.389227,10,19.200000,20.500000,6.138223,10,24.863636,27.50000,6.103650,11,24.590909,24.000000,7.002921,11,23.727273,24.500000,6.242450,11,28.590909,30.500000,5.425948,11,1.156250,0.812500,0.987553,4,0.531250,0.750000,1.022533,4,1.026786,1.875000,1.597873,7,3.225002,3.225002,0,12
9,28.384615,28.0,17.914326,13,18,18,0,13,21.405729,21.380878,6.204164,0,21.189465,21.298599,5.525985,0,24.228192,24.35923,6.132682,0,22.965544,22.941273,6.224182,0,22.807069,22.902906,5.639536,0,25.519516,25.609721,6.232449,0,0.590932,0.546352,1.166220,0,0.225834,0.254181,0.527364,0,1.983693,1.909905,1.047196,0,18.288010,18.288010,0,13
10,27.333333,27.5,16.794660,12,10,10,0,12,33.958333,32.500000,2.807930,12,33.625000,34.000000,1.367230,12,37.125000,37.00000,1.130668,12,34.791667,35.500000,2.615151,12,34.791667,35.000000,1.339239,12,38.000000,38.250000,1.314257,12,0.255208,0.531250,1.211175,12,0.526042,0.468750,0.287597,12,2.838542,2.531250,1.111810,12,0.010000,0.010000,0,12


In [11]:
#train and test data preparation
X_trn = trn_mean.loc[:,'minutes_past_mean':'Zdr_5x5_90th_count'].values
y_trn = np.log1p(trn_mean.loc[:,'Expected_mean'].values)


# Split data as training and validation set
#[X_trn, X_test, y_trn, y_test] = train_test_split(X, Y, test_size = 0.3)


In [12]:
X_trn

array([[ 29.83333333,  30.        ,  19.10410078, ...,   1.90990522,
          1.04719569,   0.        ],
       [ 29.08333333,  28.5       ,  18.72508446, ...,   0.6875    ,
          0.21568469,  12.        ],
       [ 30.75      ,  30.5       ,  17.68987898, ...,   2.0625    ,
          1.04719569,   1.        ],
       ..., 
       [ 26.5       ,  26.5       ,  18.22909762, ...,   1.90990522,
          1.04719569,   0.        ],
       [ 30.76470588,  31.        ,  18.18491618, ...,   1.90990522,
          1.04719569,   0.        ],
       [ 28.38461538,  28.        ,  18.3191087 , ...,   3.        ,
          0.21949279,   3.        ]])

In [13]:
clf = RandomForestRegressor(n_estimators=100,n_jobs=3)  # NOTE: n_jobs=-1 will use all of your cores, set to a prefered number

In [14]:
#train model
t = time.time()
clf.fit(X_trn, y_trn)
print time.time()-t

1772.57641292


In [15]:
t

1447224522.753185

In [16]:
#test performance
#y_predict = clf.predict(X_test)
#score = metrics.mean_absolute_error(y_test, np.exp(y_predict)-1) #MAE

In [17]:
#print score

In [18]:
#generate test result
test = pd.read_csv('../data/test.csv')

#selected features and Cut off outliers of Expected >= 69
index=list(test)
my_indices = [0,1,2,3,5,6,7,9,10,15,17, 18]
test_new = test[[index[i] for i in my_indices]]
#Cut off Ref values < 0
#test_new[test_new['Ref_5x5_50th']<0] = np.nan
#test_new[test_new['Ref_5x5_90th']<0] = np.nan
#test_new[test_new['RefComposite']<0] = np.nan
#test_new[test_new['RefComposite_5x5_50th']<0] = np.nan
#test_new[test_new['RefComposite_5x5_90th']<0] = np.nan
#test_new[test_new['Ref']<0] = np.nan

#combine observations with same ID by using mean
#replace Nan by overall mean
test_mean = test_new.groupby(test_new.Id).agg(['mean', 'median', 'std', 'count'])
test_mean.columns = ['_'.join(col).strip() for col in test_mean.columns.values]

index2 = list(test_mean)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
test_mean= pd.DataFrame(imp.fit_transform(test_mean),index=test_mean.index,columns=index2)

test_X =test_mean.loc[:,'minutes_past_mean':'Zdr_5x5_90th_count'].values

test_y_predict = np.exp(clf.predict(test_X))-1

In [19]:
#generate output file
#0.75 from prediction and 0.25 from marshall palmer
marshall = pd.read_csv('../data/MP_r_09.csv')

test_result_exist = pd.DataFrame()
test_result_exist['Id'] = test_mean.index
test_result_exist['Expected'] = test_y_predict

test_result = pd.DataFrame()
test_result['Id'] = test['Id'].unique()
test_result = pd.merge(test_result, test_result_exist, how='left', on=['Id'], sort=True)
test_result.loc[test_result['Expected'].isnull(), 'Expected'] = marshall.loc[test_result['Expected'].isnull(), 'Expected']
test_result.loc[test_result['Expected'].notnull(), 'Expected'] = 0.75*test_result.loc[test_result['Expected'].notnull(), 'Expected']+0.25*marshall.loc[test_result['Expected'].notnull(), 'Expected']

test_result.to_csv('../data/pks_result_v3.csv', index=False)

In [20]:
test_result

,Id,Expected
0,1,0.521423
1,2,1.236593
2,3,1.913706
3,4,4.399104
4,5,0.633064
5,6,1.046462
6,7,2.482385
7,8,1.437523
8,9,0.476100
9,10,6.228862
